In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import json
import seaborn as sn
try:
    import deephyper
    print(deephyper.__version__)
except (ImportError, ModuleNotFoundError):
    !pip install deephyper

try:
    import ray
except (ImportError, ModuleNotFoundError):
    !pip install ray

0.7.0


In [2]:
def reshape(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    return X_train, X_val, X_test

def load_data():
    (X_train, Y_train), (X_val_test, Y_val_test) = tf.keras.datasets.mnist.load_data()
    X_val, X_test = np.split(X_val_test, 2)
    Y_val, Y_test = np.split(Y_val_test, 2)
    X_train, X_val, X_test = reshape(X_train, X_val, X_test)
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [3]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = load_data()

In [4]:
def np_to_dataset(X_train, train_labels, X_test, test_labels, X_val, val_labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, train_labels))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, test_labels))
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, val_labels))
    return train_dataset, test_dataset, val_dataset

In [5]:
train_dataset, test_dataset, val_dataset = np_to_dataset(X_train, Y_train, X_test, Y_test, X_val, Y_val)


In [26]:
def batch_dataset(config: dict, train_dataset, test_dataset, val_dataset):
    batch_size = config['batch_size']
    train_dataset = train_dataset.shuffle(60000).batch(batch_size)
    test_dataset = test_dataset.shuffle(5000).batch(batch_size)
    val_dataset = val_dataset.shuffle(5000).batch(batch_size)
    return train_dataset, test_dataset, val_dataset

In [20]:
config = {
        'batch_size': 36,
        'patience': 10,
        'scheduler_factor': 0.8,
        'dropout_rate': 0.5,
        'units1': 128,
        'units2': 64,
        'units3': 32,
        'units4': 10,
        'units5': 32,
        'units6': 16,
        'activation1': 'relu',
        'activation2': 'sigmoid',
        'activation3': 'sigmoid',
        'activation4': 'softmax',
        'num_layers1': 2,
        'num_layers2': 2,
        'num_layers3': 2,
        'num_layers4': 2,
        'num_layers5': 2,
        'num_layers6': 2,
        'lr': 0.00005,
        'num_epochs': 5,
        'use_batch_norm': True,
        'optimizer': 'adam'
    }

In [8]:
import tensorflow as tf
import tensorflow.keras.backend as K
import json

def count_params(model: tf.keras.Model) -> dict:
    """Evaluate the number of parameters of a Keras model.

    Args:
        model (tf.keras.Model): a Keras model.

    Returns:
        dict: a dictionary with the number of trainable ``"num_parameters_train"`` and
        non-trainable parameters ``"num_parameters"``.
    """

    def count_or_null(p):
        try:
            return K.count_params(p)
        except:
            return 0

    num_parameters_train = int(
        np.sum([count_or_null(p) for p in model.trainable_weights])
    )
    num_parameters = int(
        np.sum([count_or_null(p) for p in model.non_trainable_weights])
    )
    return {
        "num_parameters": num_parameters,
        "num_parameters_train": num_parameters_train,
    }

In [35]:
def model(config: dict):
    input_layer = keras.layers.Input(shape=(784,))
    X = keras.layers.Dense(config["units1"], activation=config["activation1"])(input_layer)
    X = keras.layers.Dropout(config["dropout_rate"])(X)
    X = keras.layers.Dense(config["units2"], activation=config["activation2"])(X)
    X = keras.layers.Dense(config["units3"], activation=config["activation3"])(X)
    output_layer = keras.layers.Dense(config["units4"], activation=config["activation4"])(X)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model

In [33]:


model = model(config)
model.compile(
    optimizer=config["optimizer"],
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()



X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = np.reshape(X_train, (-1, 784))
X_test = np.reshape(X_test, (-1, 784))

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

model.fit(
    dataset, epochs=config["num_epochs"],
)

KeyError: 'optimizer'

In [21]:
def run(config: dict):
    X_train, Y_train, X_val, Y_val, X_test, Y_test = load_data()

    train_dataset, val_dataset, test_dataset = batch_dataset(config, X_train, Y_train, X_test, Y_test, X_val, Y_val)

    keras.layers.Input(shape=(784,))
    keras.layers.Dense(config["units1"], input_shape=(784,), activation=config["activation1"])
    keras.layers.Dropout(config["dropout_rate"])
    keras.layers.Dense(config["units2"], activation=config["activation2"])
    keras.layers.Dense(config["units3"], activation=config["activation3"])
    keras.layers.Dense(config["units4"], activation=config["activation4"])
    
    model.compile(
        optimizer=config["optimizer"],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_dataset, epochs=config["num_epochs"], validation_data=val_dataset, verbose=0
    )

    objective = history.history["val_accuracy"][-1]
    metadata = {
        "loss": history.history["loss"],
        "val_loss": history.history["val_loss"],
        "accuracy": history.history["accuracy"],
        "val_accuracy": history.history["val_accuracy"],
    }

    metadata = {k:json.dumps(v) for k,v in metadata.items()}
    metadata.update(count_params(model))

    return {"objective": objective, "metadata": metadata}

In [22]:
run(config)

2024-07-02 14:56:52.811095: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype uint8 and shape [60000,784]
	 [[{{node Placeholder/_0}}]]
2024-07-02 14:56:52.811282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2024-07-02 14:56:53.827762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [5

{'objective': 0.9336000084877014,
 'metadata': {'loss': '[1.0581825971603394, 0.587068498134613, 0.5091872811317444, 0.4797973334789276, 0.4662189781665802]',
  'val_loss': '[0.37744295597076416, 0.2940126359462738, 0.24692827463150024, 0.22580227255821228, 0.2243204563856125]',
  'accuracy': '[0.690416693687439, 0.815500020980835, 0.8431666493415833, 0.851016640663147, 0.8554666638374329]',
  'val_accuracy': '[0.8988000154495239, 0.9175999760627747, 0.9277999997138977, 0.9314000010490417, 0.9336000084877014]',
  'num_parameters': 0,
  'num_parameters_train': 111146}}

In [58]:
from deephyper.problem import HpProblem
# from deephyper.search.nas.model.space.structure import KerasStructure
# Define the search space
problem = HpProblem()

problem.add_hyperparameter((64, 256), "units1", default_value=128)
problem.add_hyperparameter((32, 128), "units2", default_value=64)
problem.add_hyperparameter((16, 64), "units3", default_value=32)
problem.add_hyperparameter((4, 32), "units4", default_value=10)
problem.add_hyperparameter((16, 64), "batch_size", default_value=32)
problem.add_hyperparameter(['relu', 'sigmoid'], "activation1", default_value='relu')
problem.add_hyperparameter(['relu', 'sigmoid'], "activation2", default_value='sigmoid')
problem.add_hyperparameter(['relu', 'sigmoid'], "activation3", default_value='sigmoid')
problem.add_hyperparameter(['softmax', 'sigmoid'], "activation4", default_value='softmax')
problem.add_hyperparameter(['adam', 'sgd'], "optimizer", default_value='adam')
problem.add_hyperparameter((0.0, 0.6), "dropout_rate", default_value=0.5)
problem.add_hyperparameter((10, 100), "num_epochs", default_value=50)

problem


Configuration space object:
  Hyperparameters:
    activation1, Type: Categorical, Choices: {relu, sigmoid}, Default: relu
    activation2, Type: Categorical, Choices: {relu, sigmoid}, Default: sigmoid
    activation3, Type: Categorical, Choices: {relu, sigmoid}, Default: sigmoid
    activation4, Type: Categorical, Choices: {softmax, sigmoid}, Default: softmax
    batch_size, Type: UniformInteger, Range: [16, 64], Default: 32
    dropout_rate, Type: UniformFloat, Range: [0.0, 0.6], Default: 0.5
    num_epochs, Type: UniformInteger, Range: [10, 100], Default: 50
    optimizer, Type: Categorical, Choices: {adam, sgd}, Default: adam
    units1, Type: UniformInteger, Range: [64, 256], Default: 128
    units2, Type: UniformInteger, Range: [32, 128], Default: 64
    units3, Type: UniformInteger, Range: [16, 64], Default: 32
    units4, Type: UniformInteger, Range: [4, 32], Default: 10

In [59]:
from tensorflow.python.client import device_lib


def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == "GPU"]


n_gpus = len(get_available_gpus())
if n_gpus > 1:
    n_gpus -= 1

is_gpu_available = n_gpus > 0

if is_gpu_available:
    print(f"{n_gpus} GPU{'s are' if n_gpus > 1 else ' is'} available.")
else:
    print("No GPU available")

No GPU available


In [60]:
import ray

# We launch the Ray run-time depending of the detected local ressources
# and execute the `run` function with the default configuration
# WARNING: in the case of GPUs it is important to follow this scheme
# to avoid multiple processes (Ray workers vs current process) to lock
# the same GPU.
if is_gpu_available:
    if not(ray.is_initialized()):
        ray.init(num_cpus=n_gpus, num_gpus=n_gpus, log_to_driver=False)

    run_default = ray.remote(num_cpus=1, num_gpus=1)(run)
    out = ray.get(run_default.remote(problem.default_configuration))
else:
    if not(ray.is_initialized()):
        ray.init(num_cpus=1, log_to_driver=False)
    run_default = run
    out = run_default(problem.default_configuration)

objective_default = out["objective"]
metadata_default = out["metadata"]

print(f"Accuracy Default Configuration:  {objective_default:.3f}")

print("Metadata Default Configuration")
for k,v in out["metadata"].items():
    print(f"\t- {k}: {v}")


2024-07-02 14:20:57.338983: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2024-07-02 14:20:57.339119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2024-07-02 14:20:58.487538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [5000,

Accuracy Default Configuration:  0.981
Metadata Default Configuration
	- loss: [0.5662249326705933, 0.2660263776779175, 0.2244025617837906, 0.2033109813928604, 0.1887427270412445, 0.17741304636001587, 0.17287389934062958, 0.16452796757221222, 0.15707066655158997, 0.15595892071723938, 0.14892783761024475, 0.14711366593837738, 0.14289157092571259, 0.14096654951572418, 0.13569745421409607, 0.1317821890115738, 0.13539882004261017, 0.130406454205513, 0.12922418117523193, 0.1279146671295166, 0.12358496338129044, 0.12500524520874023, 0.12238642573356628, 0.12483339011669159, 0.12225890159606934, 0.11522342264652252, 0.1191898062825203, 0.11746633052825928, 0.11566296964883804, 0.11472216248512268, 0.11230812221765518, 0.11197367310523987, 0.11225045472383499, 0.11071150749921799, 0.10877447575330734, 0.11051873117685318, 0.10900618135929108, 0.10886643081903458, 0.10985436290502548, 0.10907238721847534, 0.10724543780088425, 0.10592464357614517, 0.10539309680461884, 0.1045684888958931, 0.10192

In [26]:
from deephyper.evaluator import Evaluator
from deephyper.evaluator.callback import TqdmCallback

def get_evaluator(run_function):
    # Default arguments for Ray: 1 worker and 1 worker per evaluation
    method_kwargs = {
        "num_cpus": 1,
        "num_cpus_per_task": 1,
        "callbacks": [TqdmCallback()]
    }

    # If GPU devices are detected then it will create 'n_gpus' workers
    # and use 1 worker for each evaluation
    if is_gpu_available:
        method_kwargs["num_cpus"] = n_gpus
        method_kwargs["num_gpus"] = n_gpus
        method_kwargs["num_cpus_per_task"] = 1
        method_kwargs["num_gpus_per_task"] = 1

    evaluator = Evaluator.create(
        run_function,
        method="ray",
        method_kwargs=method_kwargs
    )
    print(f"Created new evaluator with {evaluator.num_workers} worker{'s' if evaluator.num_workers > 1 else ''} and config: {method_kwargs}", )

    return evaluator
evaluator_1 = get_evaluator(run)

Created new evaluator with 1 worker and config: {'num_cpus': 1, 'num_cpus_per_task': 1, 'callbacks': [<deephyper.evaluator.callback.TqdmCallback object at 0x38269ea70>]}


In [23]:
evaluator = Evaluator.create(
    run,
    method="ray",

    method_kwargs={
        "address": "auto",
        "num_cpus": n_gpus,
        "num_gpus": n_gpus,
        "num_cpus_per_task": 1,
        "num_gpus_per_task": 1,
        "num_workers": 3,
    }
)

In [27]:
from deephyper.search.hps import CBO
search = CBO(
    problem,
    evaluator_1,
    initial_points=[problem.default_configuration]
)

In [28]:
results = search.search(max_evals=25)


InvalidArgumentError: Cannot convert a Tensor of dtype variant to a NumPy array.

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(784,), activation='relu'),
    keras.layers.Dense(64, activation='sigmoid'),
    keras.layers.Dense(32, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_normal, Y_train, epochs=5)

Epoch 1/5
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/victorsu-ortiz/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/b1/89wsk5t517v75qvdm2x4ygwc0000gn/T/ipykernel_9213/3944348463.py", line 14, in <module>
    model.fit(X_train_normal, Y_train, epochs=5)
  File "/Users/victorsu-ortiz/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/var/folders/b1/89wsk5t517v75qvdm2x4ygwc0000gn/T/__autograph_generated_filezsp_59h1.py", line 15, in tf__train_function
    retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
ValueError: in user code:

    File "/Users/victorsu-ortiz/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        re

In [ ]:
model.evaluate(X_test_flat, Y_test)

313/313 [==============================] - 0s 455us/step - loss: 0.2524 - accuracy: 0.9266


[0.25240427255630493, 0.9265999794006348]

In [4]:
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = tf.keras.layers.Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(32)

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)


170498071/170498071 [==============================] - 6s 0us/step
Epoch 1/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.4983 - accuracy: 0.4542 - val_loss: 1.3734 - val_accuracy: 0.5285
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1482 - accuracy: 0.5930 - val_loss: 1.2032 - val_accuracy: 0.5726
Epoch 3/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9954 - accuracy: 0.6498 - val_loss: 0.9850 - val_accuracy: 0.6492
Epoch 4/10
1563/1563 [==============================] - 10s 7ms/step - loss: 0.8893 - accuracy: 0.6873 - val_loss: 0.9504 - val_accuracy: 0.6744
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8175 - accuracy: 0.7149 - val_loss: 0.9077 - val_accuracy: 0.6829
Epoch 6/10
1563/1563 [==============================] - 13s 8ms/step - loss: 0.7612 - accuracy: 0.7326 - val_loss: 0.8939 - val_accuracy: 0.6908
Epoch 7/10
1563/1563 [==============================] - 11s 7ms